In [2]:
#All important library imports go here
import os
import numpy as np
import cv2
from scipy.signal import find_peaks, peak_prominences
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
from google.colab import files

%matplotlib inline  
# if you are running this code in Jupyter notebook

In [3]:
#Unzipping the folder which contains the paragraph images to be segmented
#In this case, the zipped folder is uploaded to google colab in the default directory Content
#The zipped file name is files.zip
!unzip "/content/files.zip" -d "/content/"

#clearing the console after unzipping
clear_output()

In [4]:
# getting the names of all the paragraph images in the directory files
filenames=os.listdir('/content/files')
#stripping the extension of the image file from the string of the filename for further use
#Note that KHATT dataset provides the images with .tif extension
filenames_split=[filename.replace('.tif', '') for filename in filenames]

In [7]:
def directionalHistogram(img, direction='H'):
  # a function which outputs the intensity histogram for a given image along 
  #x or y directions

    (w,h) = img.shape
    sum = []
    pixel_count=0

    if(direction=='H'):
        for j in range(w-1):
          for i in range(h-1):
            pixel=img[j,i]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0

    else:
       for j in range(h-1):
          for i in range(w-1):
            pixel=img[i,j]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0

    return sum

##############################################################

def smoothHist(hist,kernel_size):
  # A function to smooth out the noise in intensity histograms of an image
  kernel = np.ones(kernel_size) / kernel_size
  return np.convolve(hist, kernel, mode='same')

##############################################################

def thresholding(image, threshold, typee='Binary', param1=0, param2=0):
  # A function to apply intensity thresholding to a grey-scale image
  # The thresholding could be simple binary thresholding or adaptive gaussian thresholding
  # If the type is not set to 'Binary' then the parameters for adaptive thresholdinf must
  # be used which are:
  #param1: local region size ( preferably an odd number)
  #param2: constant to be added to local mean
  if(typee.lower()=='binary'):
    ret, thresh= cv2.threshold(image,threshold,255,cv2.THRESH_BINARY_INV)
  else:
    thresh = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,param1,param2)
  return thresh

##############################################################

def peakinterp(interp_factor, hist, prominence_factor):
  #Given an intensity histogram of an image, this function increases the resolution of the histogram
  #by interpolation and then finds the sharp peaks in this histogram using find_peaks()
  #Interp factor controls the new resolution of the histogram
  #Prominence factor decides how much the targeted peaks stand out from the baseline of the spectrum
  resampled_pixel_space=np.linspace(0, interp_factor*len(hist)-1,interp_factor*len(hist))*(1/interp_factor)
  Original_pixel_space=np.linspace(0, len(hist)-1, len(hist))
  hist_interp = np.interp(resampled_pixel_space, Original_pixel_space, hist)
  peaks, properties = find_peaks(hist_interp, prominence=np.max(hist_interp)/prominence_factor, width=50)

  return(peaks,hist_interp, resampled_pixel_space, Original_pixel_space)

  ##############################################################

def findGradSignChange(hist_interp, resampled_pixel_space, Original_pixel_space):
    #Given an interpolated intensity histogram, this function finds the 1st derivative
    # of this histogram and outputs a vector of ones and zeros determining the sign
    # of the calculated derivative.
    # When the sign is +ve, the vector has 1
    # When the sign is -ve, the vector has 0
    hist_grad=np.gradient(hist_horizontal_smooth_interp)
    hist_grad_sign_change=np.where(hist_grad >= 0, 1, 0)
    return hist_grad_sign_change

   ##############################################################

def rle(ia):

        #A function which when given a sequence of binary values outputs the following:
        # 1) the start positions of a portion of repeated values in the sequence
        # 2) the length of the portion of repeated values
        #This will be useful in dealing with the vector representing the sign change of
        #1st derivative of image intensity histogram
       

        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]               # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)   # must include last element posi
            z = np.diff(np.append(-1, i))       # run lengths
            p = np.cumsum(np.append(0, z))[:-1] # positions
            return(z, p, ia[i])
 ##############################################################

def cutPositions(runlengths, startpositions, values, threshold,interp_factor):
  #Give a vector of ones and zeroes representing the sign change of 1st deriv. of
  # a histogram, this function smoothes out the abrupt changes in gradient sign
  # which might be an artifact of the gradient calculation.

  # This function also gives an estimation of the possible cutting locations to
  # extract lines

  viable_index=0
  for i in range(len(runlengths)):
    current_length=runlengths[i]
    if(current_length<threshold):
      values[i]=values[viable_index]
    viable_index=i

  new_hist=[]
  for i in range(len(startpositions)):
    if(values[i]):
      new_hist+=np.ones(runlengths[i]-1).tolist()
    else:
      new_hist+=np.zeros(runlengths[i]-1).tolist()

  cutpos=[]
  for i in range(1,len(startpositions)):
    last=values[i-1]
    current=values[i]
    if((last==0 and current==1)):
      cutpos.append(startpositions[i])
    elif((last==1 and i==1)):
      cutpos.append(0)


  return (cutpos, new_hist)

######################################################
def optimalThreshold(cutpos, runlengths, startpositions, values, new_hist, peaks, init_threshold, interp_factor):

  #when removing noise from the gradient sign vector prior to determining the cut locations, we use a threshold
  #value on the run lengths of ones and zeros.
  #An optimal value of the threshold is the value which when used gives us as many cut locations as detected peaks
  # in the original histogram
  while((len(cutpos)!= len(peaks))):
      init_threshold=init_threshold+interp_factor
      (cutpos, new_hist)=cutPositions(runlengths, startpositions, values, init_threshold,interp_factor)

  (cutpos, new_hist)=cutPositions(runlengths, startpositions, values, np.abs(init_threshold-interp_factor),interp_factor)
  cutpos=np.array(cutpos)/interp_factor
  
  return (cutpos, new_hist)

###################################################

def cropImageToLines(cutpos, image, direction='H'):
  (w,h)=image.shape
  cropped_images=[]
  if(direction=='H'):
    for i in range(len(cutpos)):
      currentpos=cutpos[i]
      lastpos=cutpos[i-1]
      cropped_images.append(image[lastpos:currentpos-1,0:h-1])
  else:
    for i in range(len(cutpos)):
      currentpos=cutpos[i]
      lastpos=cutpos[i-1]
      cropped_images.append(image[0:w-1, lastpos:currentpos-1])

  return cropped_images

In [9]:
#create the directory that will hold your line images
!mkdir files_segmented

#loop over all prargraph images
for m in range(len(filenames)):

    #Read the paragraph image and apply thresholding
    image = cv2.imread('/content/files/'+filenames[m],0)
    (w,h) = image.shape
    thresh1=thresholding(image, 240, typee='Binary', param1=0, param2=0)
 

    #obtaining horizontal histogram and smoothing it
    hist_horizontal=directionalHistogram(thresh1)
    hist_horizontal_smooth=smoothHist(hist_horizontal,17)

    #Obtaining peak locations from the smoothed horizontal histogram
    init_threshold=50
    interp_factor=100
    (peaks,hist_horizontal_smooth_interp,resampled_pixel_space, Original_pixel_space)=peakinterp(interp_factor, hist_horizontal_smooth, 8)
    hist_grad_sign_change=findGradSignChange(hist_horizontal_smooth_interp, resampled_pixel_space, Original_pixel_space)

    #obtaining the piecewise constant function approximating the sign change behavior of the 1st derivative of the horizontal histogram
    runlengths, startpositions, values =rle(hist_grad_sign_change)
    (cutpos, new_hist)=cutPositions(runlengths, startpositions, values, init_threshold, interp_factor)


    #Removing undesired sign changes from the piecewise function which are the result of noise or numerical artifiacts, not the desired peaks
    cutpos, new_hist=optimalThreshold(cutpos, runlengths, startpositions, values, new_hist, peaks, 50, 100)
    #displaying lines extracted from the image
    lines= cropImageToLines(cutpos.astype(int), thresh1)
    for i in range(len(lines)):
        cv2.imwrite("/content/files_segmented/"+filenames_split[m]+"_"+str(i)+".tif", lines[i])


#downloading the line image directory in zipped format
!zip -r /content/files_segmented.zip /content/files_segmented
files.download("/content/files_segmented.zip")

mkdir: cannot create directory ‘files_segmented’: File exists
  adding: content/files_segmented/ (stored 0%)
  adding: content/files_segmented/AHTD3A0161_Para1_4.tif (deflated 35%)
  adding: content/files_segmented/AHTD3A0144_Para4_2.tif (deflated 25%)
  adding: content/files_segmented/AHTD3A0158_Para1_2.tif (deflated 13%)
  adding: content/files_segmented/AHTD3A0144_Para4_3.tif (deflated 27%)
  adding: content/files_segmented/AHTD3A0168_Para1_3.tif (deflated 23%)
  adding: content/files_segmented/AHTD3A0158_Para1_3.tif (deflated 11%)
  adding: content/files_segmented/AHTD3A0161_Para1_1.tif (deflated 17%)
  adding: content/files_segmented/AHTD3A0151_Para4_0.tif (deflated 89%)
  adding: content/files_segmented/AHTD3A0168_Para1_0.tif (deflated 43%)
  adding: content/files_segmented/AHTD3A0152_Para4_0.tif (deflated 96%)
  adding: content/files_segmented/AHTD3A0144_Para4_5.tif (deflated 35%)
  adding: content/files_segmented/AHTD3A0158_Para1_1.tif (deflated 13%)
  adding: content/files_seg

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>